# 네이트 경제 뉴스 크롤링

In [ ]:
"""
주어진 날짜의 네이트 경제 뉴스를 수집하여 저장한다.
"""

import requests
from bs4 import BeautifulSoup
import time
import ujson


LIST_PAGE_URL_TMPL = "http://news.nate.com/recent?cate=eco&mid=n0301&type=c&date={}&page={}"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) " + \
             "AppleWebKit/537.36 (KHTML, like Gecko) " + \
             "Chrome/37.0.2062.94 Safari/537.36"
HEADERS = {"User-Agent": USER_AGENT}
PAGING_STOP_PAT = "뉴스가 없습니다."
SLEEP_TIME = 2


def get_target_date():
    """경제 뉴스 수집 대상 날짜를 입력받아 돌려준다."""
    
    target_date = input("네이트 경제 뉴스 수집 대상 날짜를 입력하세요(YYYYMMDD): ")
    
    return target_date
   
    
def crawl_articles(target_date):
    """주어진 수집 대상 날짜의 네이트 경제 뉴스 기사를 수집하여 저장한다."""
    
    output_file_name = gen_output_file_name(target_date)
    
    with open(output_file_name, "w", encoding="utf-8") as output_file:   
        page_num = 1
        while True:
            article_urls = fetch_article_urls(target_date, page_num)
            
            if not article_urls:
                break
                
            fetch_store_articles(output_file, article_urls)
            page_num += 1
        
        
def gen_output_file_name(target_date):
    """주어진 날짜를 이용하여 출력 파일 이름을 생성하여 돌려준다."""
    
    output_file_name = "../data/crawling/nate-economy-articles-{}.txt".format(target_date)
    
    return output_file_name
        
        
def fetch_article_urls(target_date, page_num):
    """주어진 날짜와 페이지 번호의 목록 페이지에서 기사 URL들을 수집하여 돌려준다."""

    list_page_url = gen_list_page_url(target_date, page_num)
    html = get_html(list_page_url)

    if paging_done(html):
        return None

    soup = get_soup(html)
    article_urls = ext_article_urls(soup)
    
    return article_urls


def gen_list_page_url(target_date, page_num):
    """기사 목록 페이지 URL을 생성하여 돌려준다."""
    
    list_page_url = LIST_PAGE_URL_TMPL.format(target_date, page_num)
    
    return list_page_url


def get_html(url):
    """주어진 URL에 접근하여 HTML 텍스트를 읽어서 돌려준다."""

    response = requests.get(url, headers=HEADERS)
    html = response.text
    pause()

    return html


def pause():
    """정해진 시간만큼 쉰다."""

    time.sleep(SLEEP_TIME)
    
    
def get_soup(html):
    """주어진 HTML 텍스트를 soup 객체로 만들어 돌려준다."""
    
    soup = BeautifulSoup(html, "lxml")
    
    return soup


def paging_done(html):
    """페이징이 완료되었는지를 판단한다."""

    if PAGING_STOP_PAT in html:
        return True

    return False


def ext_article_urls(soup):
    """주어진 soup 객체에서 기사 URL을 추출하여 돌려준다."""
    
    anchor_elems = soup.find_all("a", class_="lt1")
    article_urls = [elem["href"] for elem in anchor_elems]

    return article_urls


def fetch_store_articles(output_file, article_urls):
    """주어진 기사 URL의 기사를 수집하여 출력 파일에 저장한다."""
    
    for article_url in article_urls:
        html = get_html(article_url)
        soup = get_soup(html)
        title = ext_title(soup)
        date_time = ext_date_time(soup)
        body = ext_body(soup)
        write_article(output_file, title, date_time, body)   

    
def ext_title(soup):
    """주어진 soup 객체에서 기사 제목을 추출하여 돌려준다."""
    
    title_elem = soup.find("h3", class_="articleSubecjt")
    title = title_elem.string
    
    return title

            
def ext_date_time(soup):
    """주어진 soup 객체에서 기사 날짜, 시간을 추출하여 돌려준다."""
   
    outer_elem = soup.find("span", class_="firstDate")
    date_time_elem = outer_elem.find("em")
    date_time = date_time_elem.string
    
    return date_time
    
    
def ext_body(soup):
    """주어진 soup 객체에서 기사 본문을 추출하여 돌려준다."""
    
    outer_elem = soup.find("div", id="realArtcContents")
    body = outer_elem.get_text("\n", strip=True)

    return body
       

def write_article(output_file, title, date_time, body):
    """기사를 JSON 문자열로 만들어 출력 파일에 기록한다."""
    
    article = {"title": title, "date_time": date_time, "body": body}
    json_str = ujson.dumps(article, ensure_ascii=False)
    print(json_str, file=output_file, flush=True)
    print(json_str, flush=True)


def main():
    """주어진 수집 대상 날짜의 네이트 경제 뉴스 기사를 수집하여 저장한다."""
    
    target_date = get_target_date()
    crawl_articles(target_date)
    
#
# 실행
#
    
main()      